# LangChain Backend Experiment Number 2

In [16]:
import openai
from langchain.agents import Tool
import os
import dotenv
dotenv.load_dotenv()
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

## Sample Project

In [17]:
project_details="""
Basically you would put your hackathon idea in a text box
and we would run like 3 llm chains in the background
One for frontend one for backend, and then feed both chains into the last one which evaluates the two first chains if its feasible or not, and if it isn't, calls the first two chains again
The first two will also return design specifications, eg an outline of the frontend and backend
maybe another textbox in the frontend for skills/target categories
"""
project_technologies="""
OpenAI API
StreamLit
"""

### Setting Up Chain No. 1: Feature Retrieval

In [18]:

llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-3.5-turbo")
prompt = PromptTemplate(
    input_variables=["project_details", "project_technologies"],
    template="""
    Given the following project description and tech stack, identify and elaborate on the key backend features that would be necessary for development. The backend features should not involve any frontend components or styling. The backend features should be described in terms of capabilities.
    This project is a hackathon project. Break apart the features into MVP and additional features. The MVP should be the minimum features necessary to have a working prototype.
    Project description: {project_details}
    Technologies: {project_technologies}
    """
)

chain = LLMChain(llm=llm, prompt=prompt)
backend_features = chain.run({
    'project_details': project_details,
    'project_technologies': project_technologies
})
print(backend_features)

MVP Backend Features:
1. Text Processing: The backend should be able to receive the hackathon idea entered in the text box and process it using natural language processing techniques. This includes tasks such as tokenization, part-of-speech tagging, and named entity recognition.

2. LLM Chain Execution: The backend should execute three LLM (Language Model) chains in the background. One chain for the frontend, one for the backend, and one for evaluating the feasibility of the idea. The backend should handle the execution and coordination of these chains.

3. Feasibility Evaluation: The backend should evaluate the feasibility of the hackathon idea by comparing the outputs of the frontend and backend LLM chains. It should analyze the generated design specifications and determine if they align with each other. If the evaluation indicates that the idea is not feasible, the backend should trigger the execution of the frontend and backend LLM chains again.

Additional Backend Features:
1. Ski

## Setting Up Chain Two:  Feature Extraction

In [19]:
specification_prompt = PromptTemplate(
    input_variables=["backend_features", "project_technologies"],
    template="""
    Given the extracted backend features and the specified skills/technologies, create a detailed technical specification. 
    This specification should include the technologies to be used, the architecture, the different routes/endpoints, their inputs and outputs, and any potential hardware and startup costs.
    However, they should be split into two categories: MVP and additional features. The MVP should be the minimum features necessary to have a working prototype.
    You should ignore the technologies for the frontend and focus on the backend.
    Please also mention any other technical considerations.
    
    Backend Features: {backend_features}
    
    Project Technologies: {project_technologies}
    """
)

specification_chain = LLMChain(llm=llm, prompt=specification_prompt)
specification = specification_chain.run({
    'backend_features': backend_features,
    'project_technologies': project_technologies
})
print(specification)

Technical Specification:

MVP Backend Features:
1. Text Processing:
   - Technology: Natural Language Processing (NLP) libraries (e.g., NLTK, spaCy)
   - Architecture: The backend should receive the hackathon idea as input and process it using NLP techniques such as tokenization, part-of-speech tagging, and named entity recognition.
   - Inputs: Hackathon idea text
   - Outputs: Processed text with tokenized words, tagged parts of speech, and recognized named entities.

2. LLM Chain Execution:
   - Technology: Python
   - Architecture: The backend should execute three LLM chains (one for frontend, one for backend, and one for feasibility evaluation) in the background.
   - Inputs: None
   - Outputs: Generated design specifications from each LLM chain.

3. Feasibility Evaluation:
   - Technology: Python
   - Architecture: The backend should compare the outputs of the frontend and backend LLM chains to evaluate the feasibility of the hackathon idea.
   - Inputs: Generated design specific

## Setting Up Chain Three: AI Approval

In [22]:
approval_prompt = PromptTemplate(
    input_variables=["technical_specification", "aspect", "group_size", "group_experience"],
    template="""
    Given the developed technical specification, conduct a thorough review of the MVP Features only for any inconsistencies or issues. 
    Also, evaluate whether the MVP Features, can be realistically completed within the two day hackathon for {group_size} people, considering the complexity and the technology stack required.
    
    The MVP Features are specifically listed under the heading 'MVP Features'. 
    Please completely disregard any features or sections listed under 'Additional Features' or any similar headers.
    This specification is only for the {aspect} aspect of the project, and should not be evaluated for other aspects.

    Answer this question: Can the MVP Features be realistically completed within the two day hackathon for {group_size} people with this skill level: {group_experience}?
    Output only a json with keys 'approval' and 'comments'. 
    If yes, the value of 'approval' should be '1' and the value of 'comments' should be an empty string
    If not, the value of 'approval' should be '0' and the value of 'comments' should be a string with the issues and inconsistencies listed.

    Technical Specification: {technical_specification}
    
    Output only a json with keys 'approval' and 'comments'. 
    """
)
advanced_llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-4")
approval_chain = LLMChain(llm=advanced_llm, prompt=approval_prompt)
approval = approval_chain.run({
    'technical_specification': specification,
    'aspect': 'backend',
    'group_size': '4',
    'group_experience': "experienced"
})
print(approval)

{"approval": "1", "comments": ""}
